# Train a ready to use TensorFlow model with a simple pipeline

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np

# the following line is not required if BatchFlow is installed as a python package.
sys.path.append("../..")
from batchflow import Pipeline, B, C, D, F, V
from batchflow.opensets import MNIST, CIFAR10, CIFAR100
from batchflow.models.tf import ResNet18

BATCH_SIZE might be increased for modern GPUs with lots of memory (4GB and higher).

In [2]:
BATCH_SIZE = 64

# Create a dataset

[MNIST](http://yann.lecun.com/exdb/mnist/) is a dataset of handwritten digits frequently used as a baseline for machine learning tasks.

Downloading MNIST database might take a few minutes to complete.

In [3]:
dataset = MNIST(bar=True)

100%|██████████| 8/8 [00:02<00:00,  1.27it/s]


There are also predefined CIFAR10 and CIFAR100 datasets.

# Define a pipeline config

Config allows to create flexible pipelines which take parameters.

For instance, if you put a model type into config, you can run a pipeline against different models.

See [a list of available models](https://analysiscenter.github.io/batchflow/intro/tf_models.html#ready-to-use-models) to choose the one which fits you best.

In [4]:
config = dict(model=ResNet18)

# Create a template pipeline

A template pipeline is not linked to any dataset. It's just an abstract sequence of actions, so it cannot be executed, but it serves as a convenient building block.

In [5]:
train_template = (Pipeline(config=config)
                .init_variable('loss_history', init_on_each_run=list)
                .init_variable('current_loss')
                .init_model('dynamic', C('model'), 'conv_nn',
                            config={'inputs/images/shape': B('image_shape'),
                                    'inputs/labels/classes': D('num_classes'),
                                    'initial_block/inputs': 'images'})
                .to_array()
                .train_model('conv_nn', fetches='loss', images=B('images'), labels=B('labels'),
                             save_to=V('current_loss'))
                .update_variable('loss_history', V('current_loss', mode='a'))
)

# Train the model

Apply a dataset to a template pipeline to create a runnable pipeline:

In [6]:
train_pipeline = (train_template << dataset.train)

Run the pipeline (it might take from a few minutes to a few hours depending on your hardware)

In [7]:
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, drop_last=True, bar=True, prefetch=1)

100%|██████████| 937/937 [18:03<00:00,  1.40s/it]


Note that the progress bar often increments by 2 at a time - that's prefetch in action.

It does not give much here, though, since almost all time is spent in model training which is performed under a thread-lock one batch after another without any parallelism (otherwise the model would not learn anything as different batches would rewrite one another's model weights updates).

# Test the model

It is much faster than training, but if you don't have GPU it would take some patience.

In [8]:
test_pipeline = (dataset.test.p
                .import_model('conv_nn', train_pipeline)
                .init_variable('predictions') 
                .init_variable('metrics', init_on_each_run=None) 
                .to_array()
                .predict_model('conv_nn', fetches='predictions', images=B('images'), save_to=V('predictions'))
                .gather_metrics('class', targets=B('labels'), predictions=V('predictions'),
                                fmt='logits', axis=-1, save_to=V('metrics', mode='a'))
                .run(BATCH_SIZE, shuffle=True, n_epochs=1, drop_last=False, bar=True)
)

 99%|█████████▉| 156/157 [00:13<00:00, 13.39it/s]


Let's get the accumulated [metrics information](https://analysiscenter.github.io/batchflow/intro/models.html#model-metrics)

In [9]:
metrics = test_pipeline.get_variable('metrics')

Or a shorter version: `metrics = test_pipeline.v('metrics')`

Now we can easiliy calculate any metrics we need

In [10]:
metrics.evaluate('accuracy')

0.9671576433121019

In [11]:
metrics.evaluate(['false_positive_rate', 'false_negative_rate'], multiclass=None)

{'false_negative_rate': array([0.00398089, 0.        , 0.05910263, 0.02296624, 0.02156529,
        0.02501792, 0.01466485, 0.03667444, 0.13852718, 0.01652581]),
 'false_positive_rate': array([0.01185107, 0.00326121, 0.00045192, 0.0008677 , 0.0014349 ,
        0.00203828, 0.0086646 , 0.00178542, 0.00054209, 0.00543   ])}

# Save the model
After learning the model, you may need to save it. It's easy to do this.

In [12]:
train_pipeline.save_model_now('conv_nn', path='path/to/save')

## What's next?

See [the image augmentation tutorial](./06_image_augmentation.ipynb) or return to the [table of contents](./00_description.ipynb).